# Практика

In [ ]:
my_lib

In [ ]:
from my_lib import AnyName

In [1]:
class AnyName:
    def __init__(self, currency='RUB', *args, **kwargs):
        print('Метод init выполняется при обращении к классу')
        self.currency = currency
        self.price = 100
        
        # как запретить изменять атрибуты вне кода класса?
        self._api_version = 'v1'
        self.__secret_attr = 'secret'
    
    def method_1(self):
        # currency = 'RUB'  # это просто переменная
        self.currency = 'RUB'  # это атрибут
        
    def method_2(self):
        print(self.currency, self.__secret_attr)
        
    @staticmethod
    def useful_function(a, b):
        return a**2 + b**2
    
    @property
    def price_with_vat(self):
        # если валюта - рубли, то НДС 20%; в других случаях 21%
        if self.currency == 'RUB':
            return self.price * 1.2
        else:
            return self.price * 1.21

In [3]:
a = AnyName()

Метод init выполняется при обращении к классу


In [4]:
a._api_version = 'v2'
a._api_version

'v2'

In [5]:
a.method_2()

RUB secret


In [6]:
a.__secret_attr

AttributeError: ignored

In [7]:
a.price_with_vat

120.0

In [ ]:
AnyName().useful_function(4, 5)

Метод init выполняется при обращении к классу


41

In [8]:
a.useful_function(3, 4)

25

In [9]:
a.method_2()

RUB secret


In [10]:
a.method_2()

RUB secret


In [11]:
a.method_3(3, 4)

AttributeError: ignored

In [12]:
a.method_1()

In [13]:
a.currency = 'EUR'
a.currency

'EUR'

In [14]:
a.method_2()

EUR secret


In [15]:
def func_1():
    currency = 'RUB'
    return currency
    
def func_2(currency):
    print(currency)

In [16]:
result = func_1()

In [17]:
func_2(result)

RUB


# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [ ]:
import requests

In [ ]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [ ]:
m = Metrika(counter, start_date, end_date)

In [ ]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date
        
        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым
        
        self.dimensions = ''
        self.metrics = ''
        self.report = []
        
    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }
        
        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']
    
    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
        
    def browser(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'
        
        # report = self.api_request()
        report = self.full_report()
        self.report = report
    
    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report_formatted = reformatted_report
    
    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1
        
        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data
            
            offset += self.limit
            if not data or len(data) < self.limit:
                break
        
        return full_data

In [ ]:
m = Metrika()

In [ ]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11


In [ ]:
m.report

[{'dimensions': [{'name': '2020-09-08'}], 'metrics': [530.0]},
 {'dimensions': [{'name': '2020-09-09'}], 'metrics': [526.0]},
 {'dimensions': [{'name': '2020-09-07'}], 'metrics': [511.0]},
 {'dimensions': [{'name': '2020-09-01'}], 'metrics': [505.0]},
 {'dimensions': [{'name': '2020-09-02'}], 'metrics': [505.0]},
 {'dimensions': [{'name': '2020-09-03'}], 'metrics': [504.0]},
 {'dimensions': [{'name': '2020-09-10'}], 'metrics': [480.0]},
 {'dimensions': [{'name': '2020-09-04'}], 'metrics': [438.0]},
 {'dimensions': [{'name': '2020-09-05'}], 'metrics': [302.0]},
 {'dimensions': [{'name': '2020-09-06'}], 'metrics': [297.0]}]

In [ ]:
m.reformat_api_report()

In [ ]:
m.report_formatted

[['2020-09-08', 530.0],
 ['2020-09-09', 526.0],
 ['2020-09-07', 511.0],
 ['2020-09-01', 505.0],
 ['2020-09-02', 505.0],
 ['2020-09-03', 504.0],
 ['2020-09-10', 480.0],
 ['2020-09-04', 438.0],
 ['2020-09-05', 302.0],
 ['2020-09-06', 297.0]]

In [ ]:
m.browser()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161


In [ ]:
m.report

[{'dimensions': [{'name': '2020-09-08'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [329.0]},
 {'dimensions': [{'name': '2020-09-09'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [318.0]},
 {'dimensions': [{'name': '2020-09-03'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [310.0]},
 {'dimensions': [{'name': '2020-09-07'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [299.0]},
 {'dimensions': [{'name': '2020-09-02'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [298.0]},
 {'dimensions': [{'name': '2020-09-10'},
   {'icon_id': '6',
    'icon_type': 'browser',
    'name': 'Google Chrome',
    'id': '6'}],
  'metrics': [295.0]},
 {'dimensions': [{'name': '2020-09-01'},
   {'icon_i

In [ ]:
m.reformat_api_report()

In [ ]:
m.report_formatted

[['2020-09-08', 'Google Chrome', 329.0],
 ['2020-09-09', 'Google Chrome', 318.0],
 ['2020-09-03', 'Google Chrome', 310.0],
 ['2020-09-07', 'Google Chrome', 299.0],
 ['2020-09-02', 'Google Chrome', 298.0],
 ['2020-09-10', 'Google Chrome', 295.0],
 ['2020-09-01', 'Google Chrome', 292.0],
 ['2020-09-04', 'Google Chrome', 256.0],
 ['2020-09-05', 'Google Chrome', 181.0],
 ['2020-09-06', 'Google Chrome', 164.0],
 ['2020-09-07', 'Яндекс.Браузер', 58.0],
 ['2020-09-01', 'Яндекс.Браузер', 53.0],
 ['2020-09-09', 'Яндекс.Браузер', 49.0],
 ['2020-09-08', 'Яндекс.Браузер', 48.0],
 ['2020-09-02', 'Firefox', 44.0],
 ['2020-09-08', 'Firefox', 44.0],
 ['2020-09-06', 'Chrome Mobile', 43.0],
 ['2020-09-01', 'Firefox', 42.0],
 ['2020-09-09', 'Firefox', 42.0],
 ['2020-09-03', 'Яндекс.Браузер', 41.0],
 ['2020-09-04', 'Firefox', 40.0],
 ['2020-09-08', 'Chrome Mobile', 40.0],
 ['2020-09-02', 'Яндекс.Браузер', 39.0],
 ['2020-09-10', 'Chrome Mobile', 39.0],
 ['2020-09-02', 'Chrome Mobile', 38.0],
 ['2020-09-07'

### Наследование для отчетов

In [ ]:
class AbtractFabrik:
    def get_fields:
        # из каких столбцов состоит таблица
        
    def get_sources:
        # откуда вы берете данные
        
    def build:
        # выполняет вычисления
        
    def run:
        # запускают расчет
        
    def test:
        # как тестировать коррексть вашего отчета

In [ ]:
class Report_1(AbtractFabrik):
    def get_fields:
        # поля таблицы 1
        # добавляю новое поле
        
    def get_sources:
        # источники таблицы 1
        
    def build:
        # как считать ваш отчет

In [ ]:
class Report_2(AbtractFabrik):
    def get_fields:
        # поля таблицы 2
        
    def get_sources:
        # источники таблицы 2
        
    def build:
        # как считать ваш отчет

In [ ]:
class Report_3(Report_1, Report_2 ):
    def get_fields:
        # сразу получаете все поля этих отчетов как атрибуты
        
    def get_sources:
        # источники таблицы 1
        
    def build:
        # как считать ваш отчет